# Continuous Hidden Markov Models

They are useful when the observed variables don't take values in a discrete but a continuous set, like the reals. In this case, it is assumed that these variables follow a combination of Gaussian distributions, also called Gaussian Mixtured Model or GMM.


## Gaussian Mixture models

GMMs are useful to approximate distributions that are multi-modal, meaning that they have multiple modes (most common values) or "bumps". The probability distribution modelled by a GMM takes the form of a sum of weighted gaussians

$$ p(x) = \sum_i^K \tau_i \mathcal{N(\mu_i, \Sigma_i)}\,, $$

where each $\tau_k$ is a real coefficient, representing the probability that the observation $x$ belongs to the kth gaussian, and $\mathcal{N(\mu_i, \Sigma_i)}$ is a normal distribution. Note that there is a constraint here for the $\tau$ s

$$ \sum_i \tau_i = 1$$

Another way to think of all this is that there is a new hidden random variable $Z$ (hidden because it cannot be measured), following  a probability $P$, that gives the probability that $x$ belongs to the kth gaussian in the GMM. In other words

$$ \tau_k = P(Z=k) $$

Basically we are saying that there exists some $Z$ that we cannot measure but whose effect is creating the gaussian. All we can observe is the combined effect of these $Z$ s. This other point of view, in terms of $Z$, allows to place GMMs in the framework of the Expectation-Maximization algorithm.

The way to train a GMM is to first compute the responsibilities and then update the coefficients (EM-like algorithm)

$$ \gamma^{(n)}_k = p(z^{(n)}|x) = \frac{\tau_k \mathcal{N}(x^{(n)}|\mu_k, \Sigma_k)}{\sum_j \tau_j \mathcal{N}(x^{(n)}|\mu_j, \Sigma_j)}\,,$$

where $x^{(n)}$ is the n-th observation. The quantity $x^{(n)}$ could be a vector. Note that $\gamma^{(n)}_k$ just weights the importance of gaussian $k$ as compared to the contribution comming from the other gaussians in the mixture.

Then, we update the parameters following the formulas

$$ \mu_k  = \frac{1}{N_k} \sum_{n=1}^{N} \gamma_k^{(n)} x^{(n)}$$

$$ \Sigma_k = \frac{1}{N_k} \sum_{n=1}^{N} \gamma_k^{(n)} \left( x^{(n)} -\mu_k \right)\left( x^{(n)} -\mu_k \right)^T $$

$$ \tau_k = \frac{N_k}{N}\,,$$

with $N_k = \sum_{n=1}^N \gamma_k^{(n)}$.

## Back to continuous HMMs

So observations are assumed to be continuous now and we consider that they follow a GMM instead of a discrete distribution (such a categorical distribution that we saw when deriving the HMM formulas from maximum likelihood estimation in the discrete-supervised case)

$$ x_t | z_t = j \sim GMM(\{\tau^{(j)}_i, \mu^{(j)}_i, \Sigma^{(j)}_i\})\,.$$

Note that time itself is still considered to be *discrete* and that the parameters of the GMM do not change over time, i.e. they are constant (the time series of $\{x_t\}_{t=1}^T$ is covariance-stationary). Now, we also assume that the above GMM depends (is conditioned) on the hidden state $z_t$ that the system is in, so basically the hidden state alters the distribution of the corresponding $x_t$. This is the same phenomenon that ocurrs in the discrete case: the probability of the actual observed state depends on the hidden probability process.

So to model the entire dependence, we consider a matrix $R \in \mathbb{R}^{M\times K}$ that encodes the vector of $\tau$ s for each $z_j$. In particular, each row of $R$ contains the $\tau_i$ associated to the first (the hidden state space is still discrete!) hidden state. In the discrete case, we used $K$ to denote the cardinality of the observed state-space, but now this space is continuous, so $K$ does no longer represent that (arguably, the notation is a bit confusing). In reality $K$ represents the total number of gaussians that we will consider in our gaussian mixture model. The constraint  $\sum_i \tau_i = 1$ therefore translates into 

$$ \sum_i R_{ij} = 1 $$

Analogously, there is one parameter $\mu$ and $\Sigma$ per hidden state and also per gaussian ni the mixture. In addition, $x$ can be a single real-valued random variable or a vector of generic dimension $D$ and the normal distributions are multivariante gaussians. Hence, in general, we have 

$$x_t \in \mathbb{R}^D$$
$$ \mu \in \mathbb{R}^{M \times K \times D}$$
$$\Sigma \in \mathbb{R}^{M \times K \times D \times D}$$

So, now, the matrix $B$, repesenting the conditioned probability, becomes (for given hidden state $z_t=j$):

$$ p(x_t|z_t = j) \equiv B(j,t) = \sum_{k=1}^K R_{jk} \mathcal{N}(x_t, \mu_{jk}, \Sigma_{jk}) $$

We can also define each component of the GMM by means of

$$ Comp(j,k,t) = R_{jk} \mathcal{N}(x_t, \mu_{jk}, \Sigma_{jk})\,.$$

Equipped with these definitions, we can now compute the expectation-maximization algorithm.

### Expectation

The relevant equstion is

$$ \gamma(j,k,t) = \frac{\alpha(t,j)\beta(t,j)}{\sum_{j'=1}^M \alpha(t,j')\beta(t,j') } \frac{Comp(j,k,t)}{B(j,t)}